In [1]:

##########################################################
# Content:
# part I:   Install and import packages and load data
# part II:  Pre-process and vectorize the documents
# Part III: Training LDA model
# Part IV:  Find the optimal number of topics using coherence_values
# Part V:   Compute similarity of topics 
# Part VI:  Visualize the topics




In [2]:
##########################################################

##########################################################

# Part I: install and import packages and load data  

##########################################################
# please install these modules before you run the code:
!pip install gensim

!pip install nltk
#!pip install matplotlib


In [3]:

import os
import re
import pandas as pd
## `nltk.download('punkt')
import numpy as np

# NLTK
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.models import Phrases
import nltk
nltk.download('stopwords')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.matutils import kullback_leibler, jaccard, hellinger, sparse2full
#from gensim.test.utils import common_corpus
from gensim.models import LdaModel
from gensim.test.utils import datapath
#from gensim.models import LdaSeqModel
#from gensim.corpora import Dictionary, bleicorpus
#from gensim import models, similarities

# spacy for lemmatization
import spacy
from scipy.stats import wasserstein_distance

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
#matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import matplotlib.pyplot as plt
from collections import  defaultdict
from pprint import pprint


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielpele/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
############################
# Please change the working directory to your path!
# os.chdir("/Users/xinwenni/LDA-DTM/DTM") 
############################

# load data

path_datasets="/Users/danielpele/Library/CloudStorage/GoogleDrive-danpele@ase.ro/Other \
computers/Asus/G/PROIECTE/Understanding Digital Assets/Cod/Datasets/"
# load data
df = pd.read_csv(path_datasets+"posts.csv", low_memory=False)
df.head()


,date,Unnamed: 0,Page Name,User Name,Facebook Id,Page Category,Page Admin Top Country,Page Description,Page Created,Likes at Posting,...,Overperforming Score,sentiment_positive,sentiment_negative,sentiment_neutral,year,predominant_sentiment,compound_sentiment,sentiment_intensity,final_sentiment_score,compound
0,2017-01-27 23:07:20,0,Cheat Codes ATL,cheatcodesatlanta,1.000000e+14,ORG_GENERAL,US,Cheat Codes is a media company that empowers p...,25/01/2017 02:20,NaN,...,"4,283.59",0.051737,0.022936,0.925327,2017,sentiment_neutral,0.028801,14.984698,15.013499,0.028801
1,2017-12-17 00:01:18,1,Batdad,BatDadOfficial,1.000000e+14,DIGITAL_CREATOR,US,I'm BatDad,25/09/2013 01:30,6857356.0,...,184.31,0.043961,0.015284,0.940754,2013,sentiment_neutral,0.028677,13.593631,13.622308,0.028677
2,2017-12-17 02:06:24,2,John-Paul Ugwu,jpaulugwu2,1.000000e+14,PERSONAL_BLOG,NG,We are news Network,14/12/2017 12:27,NaN,...,"2,790.59",0.038404,0.225466,0.736130,2017,sentiment_neutral,-0.187062,6.258647,6.071585,-0.187062
3,2017-07-01 19:46:35,3,Bitcoin & Crypto,bitcoinandcrypto,1.000000e+14,ACTIVITY_GENERAL,US,Bringing you the latest news and analyses on t...,05/06/2017 19:19,NaN,...,251.8,0.040747,0.030607,0.928647,2017,sentiment_neutral,0.010140,10.901994,10.912135,0.010140
4,2017-12-22 05:56:21,4,LoanCoin,LoanCoin2017,1.000000e+14,TOPIC_BUSINESS_SERVICES,TH,"LOANCOIN has designed an advanced ""loan system...",02/12/2017 11:19,176368.0,...,39.22,0.090149,0.070592,0.839259,2017,sentiment_neutral,0.019557,5.525476,5.545033,0.019557


In [5]:
df[['Message']]

,Message
0,180 seconds of basic economics explained perfe...
1,Bitcoin Insta - batdadblake
2,Today is the saddest day of my life. As a Doct...
3,"Well people can stop saying ""What can you buy ..."
4,"Dear Investors, 1) Please do not share your pe..."
...,...
82805,"Bitcoin ATM scams are on the rise, and scammer..."
82806,Leaving coinbase... but staying in crypto #fin...
82807,"At AES, our main value is Safety first. Here's..."
82808,Want to accept Bitcoin and Ethereum tips on Tw...


In [6]:

##########################################################

# Part II: Pre-process and vectorize the documents

##########################################################

# Convert the 'Message' column to a list
data = df['Message'].values.tolist()
data = [str(item) for item in data]

def clean_data(data):
    # Remove Emails
    data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
    
    # Remove new line characters
    data = [re.sub('\s+', ' ', sent) for sent in data]
    
    # Remove distracting single quotes
    data = [re.sub("\'", "", sent) for sent in data]
    
    pprint(data[:1])
    return data

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def get_lemm(data):
    data_words = list(sent_to_words(data))
#    print(data_words[:1])
    
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
    
    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    
    # See trigram example
#    print(trigram_mod[bigram_mod[data_words[0]]])
    
    # Remove Stop Words
    data_words_nostops=remove_stopwords(data_words)

    # Form Bigrams
#    data_words_bigrams = make_bigrams(data_words_nostops)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    #nlp = spacy.load('en', disable=['parser', 'ner'])
    nlp = spacy.load("en_core_web_sm")
    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    
    #data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    
#    print(data_lemmatized[:1])
    return data_lemmatized

# simple clean the data first 
data=clean_data(data)
# define stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'use','also'])
nlp = spacy.load("en_core_web_sm")
#nlp = spacy.load('en', disable=['parser', 'ner'])

# Tokenize and lemmatize data 
data_lemmatized=get_lemm(data)

# Create Dictionary
id2word= corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

#Let’s see how many tokens and documents we have to train on.
print('Number of unique tokens: %d' % len(id2word))
print('Number of documents: %d' % len(corpus))



<>:13: DeprecationWarning: invalid escape sequence '\S'
<>:16: DeprecationWarning: invalid escape sequence '\s'
<>:13: DeprecationWarning: invalid escape sequence '\S'
<>:16: DeprecationWarning: invalid escape sequence '\s'
/var/folders/pk/sxdc5n5x6mb322grbvxz_nm40000gp/T/ipykernel_4417/3450740866.py:13: DeprecationWarning: invalid escape sequence '\S'
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
/var/folders/pk/sxdc5n5x6mb322grbvxz_nm40000gp/T/ipykernel_4417/3450740866.py:16: DeprecationWarning: invalid escape sequence '\s'
  data = [re.sub('\s+', ' ', sent) for sent in data]


['180 seconds of basic economics explained perfectly! Get $10 in Bitcoin FREE '
 'when you invest $100 (click our affiliate link): '
 'http://bit.ly/coinbasesign:=:https://www.coinbase.com/join/59e80734c90f890144c1f448 '
 'Inbox us if you would like anything posted on our page‼️ 👌🏾👌🏾💯💯✔️ Credit: '
 'Sean Shewmake (aka "Tommy Bottoms")']
Number of unique tokens: 61050
Number of documents: 82810


In [7]:
##########################################################

# Part III: Training LDA model 

##########################################################

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.
#
## Make a index to word dictionary.
#temp = dictionary[0]  # This is only to "load" the dictionary.
#id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)
pprint(top_topics)



Average topic coherence: -4.1269.
[([(0.016716084, 'make'),
   (0.013122917, 'go'),
   (0.011146025, 'money'),
   (0.01026498, 'get'),
   (0.009975685, 'good'),
   (0.00879592, 'take'),
   (0.0082691405, 'people'),
   (0.0076578264, 'say'),
   (0.0075277034, 'call'),
   (0.0075157243, 'work'),
   (0.0073094256, 'know'),
   (0.006598142, 'come'),
   (0.006498109, 'world'),
   (0.0063479817, 'give'),
   (0.0060525327, 'need'),
   (0.0059881657, 'year'),
   (0.005910892, 'time'),
   (0.0056609996, 'well'),
   (0.0055262637, 'many'),
   (0.005207671, 'account')],
  -1.834068231663089),
 ([(0.0786702, 'bitcoin'),
   (0.029184207, 'market'),
   (0.023897715, 'price'),
   (0.023061687, 'crypto'),
   (0.011830853, 'day'),
   (0.011721596, 'week'),
   (0.011067428, 'high'),
   (0.009543237, 'year'),
   (0.008458293, 'time'),
   (0.0077778283, 'last'),
   (0.007667661, 'today'),
   (0.0072346837, 'month'),
   (0.0066127367, 'move'),
   (0.006407395, 'see'),
   (0.006251052, 'level'),
   (0.00619

In [ ]:

##########################################################

# Part IV: find the optimal number of topics using coherence_values

##########################################################
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
#        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model = gensim.models.ldamodel.LdaModel( corpus=corpus, num_topics=num_topics, id2word=id2word,random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=15,
                                           alpha='auto',
                                           per_word_topics=True)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values


# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=1, limit=11, step=1)

# Show graph
filename='Num_Topic_CV'
limit=11; start=1; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
#plt.ylim([0.25,0.45])
#plt.legend(("coherence_values"), loc='best')
plt.savefig(filename,dpi = 720,transparent=True)
plt.show()

# optimal number is 11, then adjust the topic number, and retrain the LDA model 
num_topic=5
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topic, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=15,
                                           alpha='auto',
                                           per_word_topics=True,
                                           minimum_probability=0.0)


lda_model.show_topics()

# Save model to disk.
temp_file = datapath("lda_model")
lda_model.save(temp_file)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics(0,10))
doc_lda = lda_model[corpus]
 


In [ ]:
##########################################################

# Part V: Compute similarity of topics 

##########################################################

def plot_difference_matplotlib(mdiff, title="", annotation=None):
    """Helper function to plot difference between models.

    Uses matplotlib as the backend."""
    import matplotlib.pyplot as plt
    fig, ax = plt.subplots(figsize=(18, 14))
    data = ax.imshow(mdiff, cmap='RdBu_r', origin='lower')
    plt.title(title)
    plt.colorbar(data)

plot_difference = plot_difference_matplotlib
'''
try:
    get_ipython()
    import plotly.offline as py
except Exception:
    #
    # Fall back to matplotlib if we're not in a notebook, or if plotly is
    # unavailable for whatever reason.
    #
    plot_difference = plot_difference_matplotlib
else:
    py.init_notebook_mode()
    plot_difference = plot_difference_plotly
'''    
mdiff, annotation = lda_model.diff(lda_model, distance='hellinger', num_words=50)
plot_difference(mdiff,  annotation=annotation)
plt.tick_params(labelsize=23)
plt.savefig("topic_distance_H.png",dpi = 360,transparent=True)



In [ ]:

##########################################################

# Part VI: Visualize the topics

##########################################################
import pyLDAvis.gensim_models
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis
##########################################################

In [ ]:
pyLDAvis.save_html(vis, 'LDA_vis.html')